<a href="https://colab.research.google.com/github/Jojowaranyu/weatherwise-Waranyu-Bancherdvanich/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [13]:
# 🧪 Download packages
!pip install hands-on-ai pyinputplus

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=1420d53ea1ca92e6c9b2ad5b2f0f3a9fb84f5facdd37b19d3617eca50ddd25b1
  Stored in directory: /root/.cache/pip/wheels/85/81/c1/325c8199c5c4de8a0b9af900ff9cd8bd69853e3d9419493512
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16225 sha256=41192cebd87fffcf66beb05ccef1cbfaa1316e79f49203e8583543ca07ea1cbd
  Stored in directory: /root/.cache/pip/wheels/09/de/18/4dd2be0dbdf31870c6f0abde43b0de197b1c095655ad7da5c2
  Created wheel for stdiomask: filename=stdiomask

In [16]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'

## 📦 Setup and Configuration
Import required packages and setup environment.

In [18]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from hands_on_ai.chat import get_response

# Add any other setup code here

# Core libraries
import os
import re
from typing import Dict, Any, List, Optional, Tuple, Union
import requests
import matplotlib.pyplot as plt

os.environ["OPENWEATHER_API_KEY"] = "3a10e176bbe73766db6e4d09a4bd96d0"  # Demo key, get your own at https://openweathermap.org/api


# pyinputplus is preferred for menus, but guard if missing in some environments
try:
	import pyinputplus as pyip
except Exception:
	pyip = None  # UI cell will handle None gracefully

# Fetch weather via the teaching helper library
try:
	import fetch_my_weather as fmw
except Exception as e:
	fmw = None
	print("Warning: fetch_my_weather not installed. You can install it in Colab by running the optional pip cell above.")

# Global state used by UI/demo (kept minimal)
_LAST_LOCATION: Optional[str] = None
_LAST_FORECAST_DAYS: int = 5
_LAST_WEATHER: Optional[Dict[str, Any]] = None

# Helper for safe matplotlib usage in notebooks
plt.rcParams.update({
	"figure.figsize": (8, 4),
	"axes.grid": True,
})

## 🌤️ Weather Data Functions

In [19]:
# Define get_weather_data() function here
from typing import TypedDict

class ParsedWeather(TypedDict, total=False):
	location: str
	forecast_days: int
	error: Optional[str]
	current: Dict[str, Any]
	daily: List[Dict[str, Any]]


def _normalize_fmw_json(model_or_dict: Any) -> Dict[str, Any]:
	"""
	Convert fetch_my_weather WeatherResponse (Pydantic) or raw dict to a plain dict.
	This keeps only the keys we care about for the assignment to simplify downstream code.
	"""
	# If it's a pydantic model, use model_dump if available
	if hasattr(model_or_dict, "model_dump"):
		data = model_or_dict.model_dump()  # type: ignore[attr-defined]
	else:
		data = dict(model_or_dict)
	return data

def openWeather_getWeather(city: str, units: str = "metric"):
	api_key = os.getenv("OPENWEATHER_API_KEY")
	if not api_key:
		return None
	try:
		CITY = city
		UNITS = units  # metric for °C or "imperial" for °F
		# Step 1: Call the 5-day forecast API
		url = f"https://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={api_key}&units={UNITS}"
		response = requests.get(url)
		data = response.json()
		return data
	except Exception as e:
		print(f"Error fetching weather data: {e}")
		return None


def get_weather_data(city: str, forecast_days: int = 5):
	"""
	Retrieve weather data for a specified location.

	Args:
		location (str): City or location name
		forecast_days (int): Number of days to forecast (1-5)

	Returns:
		dict: Weather data including current conditions and forecast
	"""
	try:
		response = openWeather_getWeather(city=city, units="metric")
		return response
	except Exception as e:
		response["error"] = f"Failed to fetch weather data: {e}"
		return response

## 📊 Visualisation Functions

In [20]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
from datetime import datetime
from typing import List, Dict, Any

def _extract_day_labels(daily: List[Dict[str, Any]]) -> List[str]:
        labels: List[str] = []
        for d in daily:
                date_str = d.get("date") or d.get("astronomy", [{}])[0].get("sunrise")
                try:
                        labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
                except Exception:
                        labels.append(str(date_str))
        return labels


def create_temperature_visualisation(weather_data: Dict[str, Any], output_type: str = 'display'):
        """
        Create visualisation of temperature data.

        Args:
                weather_data (dict): The processed weather data
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """

        # get daily forecast data
        daily = weather_data.get("daily", [])
        if not daily:
                print("No daily forecast data available to plot temperatures.")
                return None

        # create x-axis max/min temperature plots label
        max_t = [int(d.get("maxtempC") or 0) for d in daily]
        min_t = [int(d.get("mintempC") or 0) for d in daily]
        labels = _extract_day_labels(daily)

        fig, ax = plt.subplots()
        ax.plot(labels, max_t, marker='o', label='Max Temp (°C)')
        ax.plot(labels, min_t, marker='o', label='Min Temp (°C)')
        ax.set_title("Daily Temperature Forecast")
        ax.set_xlabel("Day")
        ax.set_ylabel("Temperature (°C)")
        ax.legend()
        plt.xticks(rotation=0)
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None

In [21]:

def create_precipitation_visualisation(weather_data: Dict[str, Any], output_type: str = 'display'):
	"""
	Create visualisation of precipitation data.

	Args:
		weather_data (dict): The processed weather data
		output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

	Returns:
		If output_type is 'figure', returns the matplotlib figure object
		Otherwise, displays the visualisation in the notebook
	"""
	daily = weather_data.get("daily", [])
	if not daily:
		print("No daily forecast data available to plot precipitation.")
		return None

	# Some wttr.in data uses hourly precipitation; daily often contains total precipitation in mm
	# We'll use total precipitation in mm if available; otherwise, estimate from hourly
	totals: List[float] = []
	labels = _extract_day_labels(daily)

	for d in daily:
		mm = d.get("totalPrecipMM")
		if mm is None and d.get("hourly"):
			try:
				mm = sum(float(h.get("precipMM") or 0.0) for h in d["hourly"])  # type: ignore[index]
			except Exception:
				mm = 0.0
		try:
			totals.append(float(mm))
		except Exception:
			totals.append(0.0)

	fig, ax = plt.subplots()
	ax.bar(labels, totals, color="#4e79a7")
	ax.set_title("Daily Total Precipitation")
	ax.set_xlabel("Day")
	ax.set_ylabel("Precipitation (mm)")
	plt.tight_layout()

	if output_type == 'figure':
		return fig
	else:
		plt.show()
		return None

## 🤖 Natural Language Processing

In [22]:
from typing import Dict, Any

def parse_weather_question(question: str) -> Dict[str, Any]:
        """
        Parse a natural language weather question.

        Args:
                question (str): User's weather-related question

        Returns:
                dict: Extracted information including location, time period, and weather attribute
        """
        if not question or not isinstance(question, str):
                return {"error": "Invalid question provided."}

        # Basic implementation to return a dummy structure
        return {
                "raw": question,
                "location": None,
                "temperature": None,
                "uv_index": None,
                "weather_condition": None
        }

## 🧭 User Interface

In [23]:
# Define menu functions using pyinputplus or ipywidgets here
import json
def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps

def run_console_menu():
        """Simple console menu using pyinputplus if available, else fallback to input()."""
        global _LAST_LOCATION, _LAST_FORECAST_DAYS, _LAST_WEATHER

        def ask(prompt: str, choices: Optional[List[str]] = None) -> str:
                if pyip and choices:
                        return pyip.inputMenu(choices, prompt=prompt + "\n", numbered=True)
                elif pyip:
                        return pyip.inputStr(prompt)
                else:
                        # basic fallback
                        return input(prompt + (f" ({'/'.join(choices)})" if choices else "") + ": ")

        while True:
                choices = [
                        "Fetch today weather",
                        "Show temperature chart",
                        # "Show precipitation chart",
                        "Show weather summary",
                        "Ask any question",
                        "Quit",
                ]
                choice = ask("Choose an option", choices)

                if choice == choices[0]:
                        loc = ask("Enter location (e.g., Perth)")
                        days = 1
                        data = get_weather_data(loc, forecast_days=days)
                        data = get_temps(data)
                        from datetime import datetime
                        today = datetime.now().strftime("%Y-%m-%d")
                        today_temp = data.get(today)
                        average = sum(today_temp) / len(today_temp)
                        print(f"Average temperature of {loc} (celcius) is :", average)

                        _LAST_LOCATION, _LAST_FORECAST_DAYS, _LAST_WEATHER = loc, days, data

                elif choice == choices[1]:
                        if not _LAST_WEATHER:
                                print("Please fetch weather first.")
                                continue
                        # TODO : Edit function content
                        create_temperature_visualisation(_LAST_WEATHER, output_type='display')

                elif choice == choices[2]:
                        if not _LAST_WEATHER:
                                print("Please fetch weather first.")
                                continue
                        # TODO: implement weather summary function
                        create_weather_summary(_LAST_WEATHER)

                elif choice == choices[3]:
                        if not _LAST_WEATHER:
                                print("Please fetch weather first.")
                                continue
                        q = ask("Ask a weather question (e.g., 'Will it rain tomorrow in Sydney?')")
                        parsed = parse_weather_question(q)
                        # TODO : Edit function content
                        resp = generate_weather_response(parsed, _LAST_WEATHER)
                        print(resp)

                elif choice == choices[4]:
                        print("Goodbye!")
                        break

## 🧩 Main Application Logic

In [24]:
# Tie everything together here

def generate_weather_response(parsed_question: Dict[str, Any], weather_data: Dict[str, Any]) -> str:
	"""
	Generate a natural language response to a weather question.

	Args:
		parsed_question (dict): Parsed question data
		weather_data (dict): Weather data

	Returns:
		str: Natural language response
	"""
	if parsed_question.get("error"):
		return parsed_question["error"]
	if weather_data.get("error"):
		return weather_data["error"]

	location = weather_data.get("location") or parsed_question.get("location") or "the selected location"
	current = weather_data.get("current", {})
	daily = weather_data.get("daily", [])

	attribute = parsed_question.get("attribute")
	time_spec = parsed_question.get("time")

	def format_day(d: Dict[str, Any]) -> str:
		date = d.get("date", "")
		desc = (d.get("hourly", [{}])[0].get("weatherDesc") or [{"value": ""}])[0].get("value", "")
		return f"{date} ({desc})"

	if attribute == "temperature":
		# If today requested or unspecified, use current
		if time_spec in (None, 0):
			t = current.get("temp_C")
			feels = current.get("FeelsLikeC") or current.get("FeelsLikeC")
			desc = (current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")
			if t is not None:
				return f"Right now in {location}: {t}°C, {desc}. Feels like {feels}°C."
		# Otherwise use daily forecast
		if isinstance(time_spec, int) and 0 <= time_spec < len(daily):
			d = daily[time_spec]
			return (
				f"In {location} on {d.get('date')}: min {d.get('mintempC')}°C, max {d.get('maxtempC')}°C."
			)
		if isinstance(time_spec, slice):
			parts = []
			for d in daily[time_spec]:
				parts.append(f"{d.get('date')}: {d.get('mintempC')}–{d.get('maxtempC')}°C")
			if parts:
				return f"Temperature forecast for {location}: " + "; ".join(parts)

	if attribute == "precipitation":
		if isinstance(time_spec, int) and 0 <= time_spec < len(daily):
			d = daily[time_spec]
			mm = d.get("totalPrecipMM")
			return f"Expected precipitation in {location} on {d.get('date')}: {mm} mm."
		if time_spec in (None, 0):
			# Try current precipitation
			precip_mm = current.get("precipMM")
			if precip_mm is not None:
				return f"Right now in {location}, precipitation is {precip_mm} mm."
		if isinstance(time_spec, slice):
			parts = []
			for d in daily[time_spec]:
				parts.append(f"{d.get('date')}: {d.get('totalPrecipMM', 0)} mm")
			if parts:
				return f"Precipitation forecast for {location}: " + "; ".join(parts)

	if attribute == "wind":
		if time_spec in (None, 0):
			spd = current.get("windspeedKmph")
			dirp = current.get("winddir16Point")
			if spd is not None:
				return f"Current wind in {location}: {spd} km/h {dirp or ''}."

	# Fallback generic summary
	if current:
		desc = (current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")
		t = current.get("temp_C")
		return f"Currently in {location}: {t}°C, {desc}."

	return "I'm unable to summarise the weather from the provided data."

## 🧪 Testing and Examples

In [ ]:
run_console_menu()

Choose an option
1. Fetch today weather
2. Show temperature chart
3. Show weather summary
4. Ask any question
5. Quit
1
Enter location (e.g., Perth)Perth
Forecast days (1-5)3
Average temperature of Perth (celcius) is : 16.8875
Choose an option
1. Fetch today weather
2. Show temperature chart
3. Show weather summary
4. Ask any question
5. Quit


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.